# SB125 Fund Split Analysis

## Question:
- How did RTAs split SB125 funds between operations and capital?

## Methodology:
- upload all avilable `SB125 fund request template` files to gcs
- examine all files for consistencies:
    - come with cleaning plan for inconsistent examples (files withot capital/operating columns)
- concat all rows across all files


## Notes:
- some RTPAs did not submit a `SB125 fund request template.xlsx` file, but instead included an quivilent file their allocation package

In [1]:
import pandas as pd
import os

GCS_PATH = "gs://calitp-analytics-data/data-analyses/sb125/fund_split/"

In [8]:
file_list = [
    "sierra_fund_request.xlsx",
    "slocog_fund_request.xlsx",
    "tehema_fund_request.xlsx",
    "tuolumne_fund_request.xlsx",
    "ventura_fund_request.xlsx",
    "alpine_fund_request.xlsx",
    "amador_fund_request.xlsx",
    "butte_fund_request.xlsx",
    "calaveras_fund_request.xlsx",
    "del_norte_fund_request.xlsx",
    "el_dorado_fund_request.xlsx",
    "humboldt_fund_request.xlsx",
    "kern_fund_request.xlsx",
    "kings_fund_request.xlsx",
    "la_metro_fund_request.xlsx",
    "lake_fund_request.xlsx",
    "lassen_fund_request.xlsx",
    "madera_fund_request.xlsx",
    "mariposa_fund_request.xlsx",
    "mendocino_fund_request.xlsx",
    "merced_fund_request.xlsx",
    "mtc_fund_request.xlsx",
    "nevada_fund_request.xlsx",
    "orange_fund_request.xlsx",
    "placer_fund_request.xlsx",
    "plumas_fund_request.xlsx",
    "riverside_fund_request.xlsx",
    "san_benito_fund_request.xlsx",
    "san_diego_mts_fund_request.xlsx",
    "santa_cruz_fund_request.xlsx",
    "shasta_fund_request.xlsx",
]

file_list.sort()

In [10]:
file_list

['alpine_fund_request.xlsx',
 'amador_fund_request.xlsx',
 'butte_fund_request.xlsx',
 'calaveras_fund_request.xlsx',
 'del_norte_fund_request.xlsx',
 'el_dorado_fund_request.xlsx',
 'humboldt_fund_request.xlsx',
 'kern_fund_request.xlsx',
 'kings_fund_request.xlsx',
 'la_metro_fund_request.xlsx',
 'lake_fund_request.xlsx',
 'lassen_fund_request.xlsx',
 'madera_fund_request.xlsx',
 'mariposa_fund_request.xlsx',
 'mendocino_fund_request.xlsx',
 'merced_fund_request.xlsx',
 'mtc_fund_request.xlsx',
 'nevada_fund_request.xlsx',
 'orange_fund_request.xlsx',
 'placer_fund_request.xlsx',
 'plumas_fund_request.xlsx',
 'riverside_fund_request.xlsx',
 'san_benito_fund_request.xlsx',
 'san_diego_mts_fund_request.xlsx',
 'santa_cruz_fund_request.xlsx',
 'shasta_fund_request.xlsx',
 'sierra_fund_request.xlsx',
 'slocog_fund_request.xlsx',
 'tehema_fund_request.xlsx',
 'tuolumne_fund_request.xlsx',
 'ventura_fund_request.xlsx']

In [96]:
def clean_fund_request(file:str) -> pd.DataFrame:
    """
    reads in the file from GCS, maps col_names list to df columns, drops all the blank rows.
    returns df.
    """
    col_names = [
    "RTPA",
    "Implementing Agenc-y/-ies",
    "Project",
    "Fund Source",
    "capital_FY23-24",
    "capital_FY24-25",
    "capital_FY25-26",
    "capital_FY26-27",
    "operating_FY23-24",
    "operating_FY24-25",
    "operating_FY25-26",
    "operating_FY26-27",
    "total",
]
    
    df = pd.read_excel(f"{GCS_PATH}{file}", header=2, nrows=40, names=col_names).drop(columns="total")
    row_drop = df["RTPA"].isin(["Grand Total", "RTPA"])
    df = df.drop(df[row_drop].index)
    df = df.dropna(how= "all")
    df[["RTPA", "Implementing Agenc-y/-ies","Project"]] = df[["RTPA", "Implementing Agenc-y/-ies","Project"]].ffill()
    
    return df


In [72]:
alpine = clean_fund_request("alpine_fund_request.xlsx")

In [73]:
len(alpine.columns)

12

In [74]:
amador = clean_fund_request("amador_fund_request.xlsx")

In [75]:
display(alpine.dtypes, 
        amador.dtypes,
)

RTPA                          object
Implementing Agenc-y/-ies     object
Project                       object
Fund Source                   object
capital_FY23-24              float64
capital_FY24-25              float64
capital_FY25-26              float64
capital_FY26-27              float64
operating_FY23-24            float64
operating_FY24-25            float64
operating_FY25-26            float64
operating_FY26-27            float64
dtype: object

RTPA                          object
Implementing Agenc-y/-ies     object
Project                      float64
Fund Source                   object
capital_FY23-24              float64
capital_FY24-25              float64
capital_FY25-26              float64
capital_FY26-27              float64
operating_FY23-24            float64
operating_FY24-25            float64
operating_FY25-26            float64
operating_FY26-27            float64
dtype: object

first iteration of fund_request_checker func.
>def read_in(file:str) -> pd.DataFrame:
>    df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>    df = df.dropna(how= "all")
>   
>    if len(df.columns) == 13:
>        print(f"{file} can use clean_fund_request func.")
>    else: 
>        print(f"needs manual check, {file}")
       

old iteration
>def fund_request_checker(file_list:list) -> pd.DataFrame:
>
>    for file in file_list:
>    
>        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>        df = df.dropna(how= "all")
>    
>        if len(df.columns) == 13:
>            print(f"{file} can use clean_fund_request func.")
>        else: 
>            print(f"needs manual check, {file}")
       

In [ ]:
#fund_request_checker(file_list)

old iteration
>def fund_request_checker_v2(file_list:list):
>    gtg_files = []
>    manual_review = []
>    for file in file_list:
>    
>        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
>        df = df.dropna(how= "all")
>    
>        if len(df.columns) == 13:
>            gtg_files.append(f"{file}")
>        else: 
>            manual_review.append(f"{file}")
>    return display(
>        "good to go files",
>        list(gtg_files), 
>        "needs manual check", 
>        list(manual_review)
>    )

In [ ]:
#fund_request_checker_v2(file_list)

In [52]:
def fund_request_checker_v3(file_list:list) -> tuple:
    """takes in list of fund_request excel file name. reads in each file, checks if DF has 13 columns.
    if yes, appends do good-to-go list. else, appends to needs-manual-review.
    output is a tuple of the 2 list.
    assign 2 variables to use this func.
    """
    gtg_files = []
    manual_review = []
    for file in file_list:
    
        df = pd.read_excel(f"{GCS_PATH}{file}", nrows=40)
        df = df.dropna(how= "all")
    
        if len(df.columns) == 13:
            gtg_files.append(f"{file}")
        else: 
            manual_review.append(f"{file}")
    return gtg_files, manual_review

In [93]:
good_list, review_list = fund_request_checker_v3(file_list)

In [55]:
display(
    len(good_list),
    len(review_list)
)

24

7

In [61]:
def cleaner_loop(gtg_list:list) -> dict: 
    """
    takes in good-to-go list from fund_request_checker.
    applies the clean_fund_request function to each item on the list, then appends to dictionary.
    key is name of the file, value is the cleaned dataframe.
    output is dictionary. 
    """
    cleaned_df ={}

    for name in gtg_list:
        cleaned_df[name] = clean_fund_request(name)
    return cleaned_df

In [97]:
cleaned_fund_request = cleaner_loop(good_list)

In [90]:
display(
    type(cleaned_fund_request),
    len(cleaned_fund_request),
    list(cleaned_fund_request.keys()),
)

dict

24

['alpine_fund_request.xlsx',
 'amador_fund_request.xlsx',
 'calaveras_fund_request.xlsx',
 'del_norte_fund_request.xlsx',
 'el_dorado_fund_request.xlsx',
 'humboldt_fund_request.xlsx',
 'kings_fund_request.xlsx',
 'la_metro_fund_request.xlsx',
 'lake_fund_request.xlsx',
 'madera_fund_request.xlsx',
 'mariposa_fund_request.xlsx',
 'mendocino_fund_request.xlsx',
 'merced_fund_request.xlsx',
 'nevada_fund_request.xlsx',
 'placer_fund_request.xlsx',
 'plumas_fund_request.xlsx',
 'riverside_fund_request.xlsx',
 'san_benito_fund_request.xlsx',
 'san_diego_mts_fund_request.xlsx',
 'shasta_fund_request.xlsx',
 'sierra_fund_request.xlsx',
 'slocog_fund_request.xlsx',
 'tehema_fund_request.xlsx',
 'tuolumne_fund_request.xlsx']

In [98]:
# view all the good-to-go df

from IPython.display import display

# .items() creates tuples of each element in the dict. key:value maps to key:df

for key, df in cleaned_fund_request.items():
    print(f"DataFrame: {key}")
    display(df)


DataFrame: alpine_fund_request.xlsx


,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,360641.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,TIRCP,NaN,360794.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (GGRF),3616.684,3781.0,3781.0,3781.0,NaN,NaN,NaN,NaN
3,Alpine County Transportation Commission,Alpine County Transportation Commission,Transit Facility Conversion Project,ZETCP (PTA),3123.316,NaN,NaN,NaN,NaN,NaN,NaN,NaN


DataFrame: amador_fund_request.xlsx


,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
2,Amador County Transportation Commission,Amador County Transportation Commission,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Amador County Transportation Commission,Amador Transit,NaN,TIRCP,100000.0,75000.0,NaN,NaN,NaN,NaN,NaN,NaN


DataFrame: calaveras_fund_request.xlsx


...

,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,SLOCOG,City of Morro Bay,City of MB-1,TIRCP,124000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SLOCOG,City of Morro Bay,City of MB-2,ZETCP (GGRF),NaN,NaN,800000.0,NaN,NaN,NaN,NaN,NaN
2,SLOCOG,City of San Luis Obispo,City of SLO-1,TIRCP,280000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SLOCOG,City of San Luis Obispo,City of SLO-2,TIRCP,395000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SLOCOG,City of San Luis Obispo,City of SLO-3,TIRCP,NaN,130000.0,NaN,NaN,NaN,NaN,NaN,NaN
5,SLOCOG,City of San Luis Obispo,City of SLO-4,TIRCP,NaN,750000.0,NaN,NaN,NaN,NaN,NaN,NaN
6,SLOCOG,San Luis Obispo RTA,RTA-1,TIRCP,1778000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,SLOCOG,San Luis Obispo RTA,RTA-2,TIRCP,500000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SLOCOG,San Luis Obispo RTA,RTA-3,ZETCP (PTA),200000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,SLOCOG,San Luis Obispo RTA,RTA-4,TIRCP,1400000.0,3275000.0,NaN,NaN,NaN,NaN,NaN,NaN


DataFrame: tehema_fund_request.xlsx


,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,Tehama County Transportation Commission,Tehama County Transit Agency,Administration Share,ZETCP (PTA),8.115100e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Tehama County Transportation Commission,Tehama County Transit Agency,Project Development- Facility Conversion,TIRCP,3.000000e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tehama County Transportation Commission,Tehama County Transit Agency,Transit Facility Infrastructure Project,TIRCP,3.287498e+06,3595798.0,NaN,NaN,NaN,NaN,NaN,NaN
3,Tehama County Transportation Commission,Tehama County Transit Agency,Transit Facility Infrastructure Project,ZETCP (PTA),7.979848e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Tehama County Transportation Commission,Tehama County Transit Agency,Transit Facility Infrastructure Project,ZETCP (GGRF),1.863735e+05,194840.0,194840.0,194840.0,NaN,NaN,NaN,NaN


DataFrame: tuolumne_fund_request.xlsx


,RTPA,Implementing Agenc-y/-ies,Project,Fund Source,capital_FY23-24,capital_FY24-25,capital_FY25-26,capital_FY26-27,operating_FY23-24,operating_FY24-25,operating_FY25-26,operating_FY26-27
0,Tuolumne County Transportation Council,Tuolumne County Transit Agency,Operating/SRTP/Implementation,TIRCP,NaN,NaN,NaN,NaN,439857.02,704157.19,704157.1900,704224.1900
1,Tuolumne County Transportation Council,Tuolumne County Transit Agency,Zero Emission Transit Infrastructure,TIRCP,200000.0,955441.333333,955441.333333,955441.333333,34955.50,34955.50,34955.5000,34955.5000
2,Tuolumne County Transportation Council,Tuolumne County Transit Agency,Via Software,TIRCP,218366.0,NaN,NaN,NaN,10389.66,140378.00,147471.1403,155639.2157
3,Tuolumne County Transportation Council,Tuolumne County Transit Agency,Zero Emission Buses,ZETCP (GGRF),NaN,208000.000000,208000.000000,208000.000000,NaN,2091.00,2091.0000,2091.0000
